### Purpose
This notebook serves to contact Pytrends API and retrieve relative popularities

In [4]:
import pandas as pd
from pytrends.request import TrendReq
import time
from time import sleep
from random import randint
import random
import calendar
import time
import random
import csv

In [ ]:
pytrends = TrendReq(timeout=None, retries=10, backoff_factor=0.5)
batch_size = 4
predefined_element = "air travel"
unique_tags = []
i = -4 # Initial value is -4
## TODO Get your keywords from the csv files you have saved
#file_path = r"C:\School\Semester_1\Data_Wrangling\Data_in_the_wild_exam\data\intermediate\pytrends\auxiliary\2022\missing_pytrends_tags_2022.csv"
#df = pd.read_csv(file_path, sep=';')
#for ind in df.index:
#    unique_tags.append(df['tag'][ind])
unique_tags = ['horses', 'horse']
while i <= len(unique_tags) - 1:
    try:
        i += batch_size
        if i + batch_size > len(unique_tags) - 1:
             batch = unique_tags[i : len(unique_tags) - 1]
        else:
            batch = unique_tags[i : i + batch_size]
        # Add the pre-defined element
        batch.append(predefined_element)
        
        print(f'This is my batch {i} - {i+4}'.format(i=i))
        ## TODO Replace with your parameters
        pytrends.build_payload(batch, timeframe='2023-01-01 2023-01-31', geo='GB')
        response = pytrends.interest_over_time()

        current_GMT = time.gmtime()
        time_stamp = calendar.timegm(current_GMT)
        ## TODO replace with your path
        response.to_csv(f'C:\School\Semester_1\Data_Wrangling\Data_in_the_wild_exam\data\\raw\pytrends\\raw_results_2022\{time_stamp}_keywords.csv', sep=";")

        # Wait a little so I don't overwhelm the API with requests
        sleep(randint(5, 18))
        
        if i + batch_size > len(unique_tags) - 1:
            raise KeyError('done')
        
    except KeyError as error:
        print('Done! You can empty the leftover csv!')
        with open(r"C:\School\Semester_1\Data_Wrangling\Data_in_the_wild_exam\data\intermediate\pytrends\auxiliary\2022\leftover_tags.csv", "x", encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Empty! We are done!'])  # Write header

    ## It is common to get 429 exceptions from the API. In case we do get one,
    # we will interrupt the scraping for a while, saving our progress in a csv
    # After a while we shuffle the remaining tags to ensure that the request is random
    # This is done so that we don't repeat the same request, which is seemingly treated as suspicious by Google    
    except Exception as error:
            print(f'The request failed at batch {i} - {i+4}'.format(i=i))
            print('Noting the last index of the failed request')
            display(error)

            # Write the index to the file
            with open(r"C:\School\Semester_1\Data_Wrangling\Data_in_the_wild_exam\data\intermediate\pytrends\auxiliary\2022\failed_requests_index.txt", "x") as file:
                file.write(str(i))

            # Slice the unique_tags array based on the index
            unique_tags = unique_tags[i:]
            print(unique_tags)
            # TODO replace with your path
            with open(r"C:\School\Semester_1\Data_Wrangling\Data_in_the_wild_exam\data\intermediate\pytrends\auxiliary\2022\leftover_tags.csv", "w") as file:
                writer = csv.writer(file)
                writer.writerow(["tag"])  # Write header
                for tag in unique_tags:
                    writer.writerow([tag])

            #Reset i == -4.
            i = -4

            #Shuffle the array so we send random elements to confuse the API
            random.shuffle(unique_tags)
            sleep(randint(10, 15))

            continue

### For testing
Some code with dummy arrays to simulate the requests

In [ ]:
checksum = 0
pytrends = TrendReq(timeout=None, retries=10, backoff_factor=0.5)
batch_size = 4
predefined_element = "air travel"
unique_tags = []
i = -4 # modify this value depending on progress
file_path = r"C:\School\Semester_1\Data_Wrangling\Data_in_the_wild_exam\data\intermediate\pytrends\auxiliary\2022\missing_pytrends_tags_2022.csv"
df = pd.read_csv(file_path, sep=';')
for ind in df.index:
    unique_tags.append(df['tag'][ind])
print(unique_tags)
while i <= len(unique_tags) - 1:
    try:
        i += batch_size
        checksum += 1
        print(i, len(unique_tags))
        if i + batch_size > len(unique_tags) - 1:
             batch = unique_tags[i : len(unique_tags) - 1]
        else:
            batch = unique_tags[i : i + batch_size]
        # Add the pre-defined element
        batch.append(predefined_element)
        
        print(f'This is my batch {i} - {i+4}'.format(i=i))
        print(batch)

        # Wait a little so I don't overwhelm the API with requests
        #make request
        #make the csv file with response
        #sleep(randint(5, 18))
        print('Sending request and simulating sleep!! ZZzzzzzZZZzzz')

        if i + batch_size > len(unique_tags) - 1:
            raise KeyError('done')

        if i > 10:
            raise Exception('Dummy exception at third req!!')
        
        
    except KeyError as error:
        print('Done! You can empty the leftover csv!')
        with open("leftover_tags.csv", "w", encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Empty! We are done!'])  # Write header
        
    except Exception as error:
            print(f'The request failed at batch {i} - {i+4}'.format(i=i))
            print('Noting the last index of the failed request')

            # Write the index to the file
            with open("failed_requests_index.txt", "w") as file:
                file.write(str(i))

            # Slice the unique_tags array based on the index
            unique_tags = unique_tags[i:]
            print(unique_tags)

            with open("leftover_tags.csv", "w", encoding='utf-8', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["tag"])  # Write header
                for tag in unique_tags:
                    writer.writerow([tag])

            #Reset i == -4.
            i = -4

            #Shuffle the array so we send random elements to confuse the API
            random.shuffle(unique_tags)

            continue
    
